# AKSSF Anadromous Waters Catalog Data Join
Use spatial join to add AWC_CODE, Species and LSTAGE information to AKSSF catchments.  Add new field to concatenate species+lstage and field ot designate salmon spawning habitat presence or absence.

In [1]:
import os, arcpy, sys,datetime, traceback
import time

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\A

## Functions

In [3]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

## Section 1
### Set input datasets and output locations
User to input paths for necessary input data and output locations



In [4]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}# Specify path to AKSSF parent directory
while True:
    try:
        userinput = replace_all((input('Input path to AWC HUC12 catchments outlets fc.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_AWC_HUC12_outlet_cats_merge') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_AWC_HUC12_outlet_cats_merge"),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            catsint = userinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AKSSF AWC HUC12 outlet catchments set to {catsint}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events_source = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC events feature class set to {awc_events_source}\n {"-"*100}')

# Specify path to AWC Lakes events fc
while True:
    try:
        userinput4 = replace_all((input('Input path to awc lakes events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents"), inputDict)
        if not arcpy.Exists(userinput4):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_lakes = userinput4
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC lakes events feature class set to {awc_lakes}\n {"-"*100}')

# Specify path to output gdb
while True:
    try:
        userinput3 = replace_all((input('Input path to output gdb.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb"), inputDict)
        if not arcpy.Exists(userinput3):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            outputgdb = userinput3
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'Output GDB set to {outputgdb}\n {"-"*100}')

# Specify path to sites spatial join fc
while True:
    try:
        siteinput = replace_all((input('Input path to temp sites fc.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points"),inputDict)
        if not arcpy.Exists(siteinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            sites = siteinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AKSSF AWC HUC12 catchment outlet centroids {sites}\n {"-"*100}')



AKSSF AWC HUC12 outlet catchments set to U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\AKSSF_AWC_HUC12_outlet_cats_merge
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
AWC lakes events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\AWCLakeEvents
 ----------------------------------------------------------------------------------------------------
Output GDB set to U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb
 ----------------------------------------------------------------------------------------------------
AKSSF AWC HUC12 catchment outlet centroids U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\AKSSF_awcHuc12_outlet_cats_points
 -------------------------------------------------------------------------------------------------

## Join awc data to catchments via spatial join
* During spatial join add new field 'AWC_OID' and use match_type = join on awc=Events ObjectID so sp/ls can be queried from source polyline, not from AWC_CODE
    * Must copy to memory and second oid field to merge because source OID field will not merge with join option - just copies OID from output fc to this joined field?
* Add field 'sp_ls' to record species and l_stage comma seperated concatenations and field 'sa_sp_hab' to record presence absence of salmon spawning habitat

In [5]:
# Make copy of awcEvents in memory and add second OID field because original field will not honor the merge rule 'join' in arcpy spatial join (works in Pro?)
awc_events = arcpy.FeatureClassToFeatureClass_conversion(awc_events_source,r'memory','awcEventsCopy')
arcpy.AddField_management(awc_events,'OID2','LONG')
with arcpy.da.UpdateCursor(awc_events,['OBJECTID','OID2']) as cur:
    for row in cur:
        row[1] = row[0]
        cur.updateRow(row)
    del(row)
del(cur)
print(f'AWC Copy complete\n{"*"*100}')

AWC Copy complete
****************************************************************************************************


In [6]:
import time, datetime
arcpy.env.overwriteOutput = True
# Parameters
joinopt = "JOIN_ONE_TO_ONE"
jointype = "KEEP_ALL"
matchopt = "INTERSECT"
s_rad = "1 Centimeters"
processStart = time.time()

print(f'Joining species and lifestage data from {awc_events} to {catsint}')
awcjoin = arcpy.analysis.SpatialJoin(catsint,
                                     awc_events,
                                     os.path.join(outputgdb,'AKSSF_all_cats_int_awc_spJoin'),
                                     joinopt,
                                     jointype,
                                     r'cat_ID_con "cat_ID_con" true true false 255 Text 0 0,First,#,catsint,cat_ID_con,0,255;'
                                     r'MERGE_SRC "MERGE_SRC" true true false 255 Text 0 0,First,#,catsint,MERGE_SRC,0,255;'
                                     # Add AWC_OID field so Sp/Ls records can be queried from polyine rather than AWC_CODE
                                     r'AWC_OID "AWC_OID" true true false 500 Text 0 0,Join,",",awcEventArcs,OID2,-1,-1;'
                                     r'AWC_CODE "AWC_CODE" true false false 500 Text 0 0,Join,",",awc_events,AWC_CODE,0,70;'
                                     r'SPECIES "SPECIES" true false false 500 Text 0 0,Join,",",awc_events,SPECIES,0,2;'
                                     r'LSTAGE "LSTAGE" true false false 500 Text 0 0,Join,",",awc_events,LSTAGE,0,1',
                                     matchopt,
                                     s_rad,
                                     ''
                                     )

# ADD additional fields
arcpy.AddField_management(awcjoin,'sp_ls','TEXT')
arcpy.AddField_management(awcjoin,'sa_sp_hab','SHORT')
arcpy.AddField_management(awcjoin,'sa_sp_count','SHORT')
arcpy.AddField_management(awcjoin,'mult_awc','SHORT')
arcpy.AddField_management(awcjoin,'AWC_CODE_qc','TEXT',"","",500)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')


Joining species and lifestage data from memory\awcEventsCopy to U:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\AKSSF_AWC_HUC12_outlet_cats_merge
Process complete
Process completed at 2022-03-29 01:15 (Elapsed time: 0:03:26)
****************************************************************************************************



## ~~Make dictionary from awc events~~
## Cannot use a dictionary because the awc records from spatial join do not match records in value Dictionary for all systems.



In [8]:
awcDict = {}
with arcpy.da.SearchCursor(awc_events, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcDict,row[0],sp_ls_val)

awcOidDict = {}
with arcpy.da.SearchCursor(awc_events, ['OBJECTID',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcOidDict,row[0],sp_ls_val)

awcLakesDict = {}
with arcpy.da.SearchCursor(awc_lakes, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcLakesDict,row[0],sp_ls_val)
print(f'AWC Dictionaries Created\n{("*"*100)}')


AWC Dictionaries Created
****************************************************************************************************


## Use update cursor to populate new field
sa_sp_hab: 1 = spawning, 2 = salmon species present but not spawning, null = no awc records
sa_sp_count: # of salmon species listed as spawning, null for no data
mult_awc: 1 for multiple awc codes, 0 for one awc code and null for no data

In [9]:
awcjoin = os.path.join(outputgdb,'AKSSF_outlet_cats_awc_spJoin')
dropfields = ['Join_Count', 'NHDPlusID','catID','Shape','TARGET_FID','SOURCEFC','region']
keepfields = [f.name for f in arcpy.ListFields(awcjoin) if f.name not in dropfields]
sa_sp_codes = ['CO_s','S_s','K_s', 'CH_s','P_s']
sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
print(keepfields)
multirecslist = []
multirecsDict = {}
norecslist = []
sp_lsList = []
awcCodeDict = {}
catAwcDict = {}
curfields = ['cat_ID_con','AWC_CODE','Species', 'LSTAGE', 'sp_ls','sa_sp_hab','mult_awc','AWC_CODE_qc','AWC_OID','sa_sp_count']
c2 = 0
with arcpy.da.UpdateCursor(awcjoin, curfields) as cur:
    for row in cur:
        awcvals = []
        print(f'{(100*"*")}')
        if row[1] is not None:
            for val in row[1].split(','):
                val2 = "'"+str(val)+"'"
                awcvals.append(val)
            awcvals = list(set(awcvals))
            if len(str(awcvals).split(',')) > 1:
                print(f'Multiple awc records({len(str(awcvals).split(","))}) for this catchment ')
                append_value(multirecsDict,row[0],awcvals)
                row[6] = 1
                row[7] = str(awcvals).strip("[]").replace("'","")
            else:
                row[7]=awcvals[0]
                row[6] = 0
            spvals = row[2].split(",")
            lsvals = row[3].split(",")
            snum = len(row[2].split(","))
            cat_spList =[]
            for i in range(snum):
                sp_ls = spvals[i] + '_' + lsvals[i]
                append_value(awcCodeDict,row[1],sp_ls)
                append_value(catAwcDict,row[0],sp_ls)
                sp_lsList.append(sp_ls)
                cat_spList.append(sp_ls)
            print(f'{row[0]} has {snum} species_ls records {cat_spList}')
            print(f'list without duplicate records {list(set(cat_spList))}')
            row[4] = str(list(set(cat_spList))).strip("[]").replace("'","")
            if len(set(cat_spList).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {set(cat_spList).intersection(sa_sp_codes)} salmon spawning records')
                saspcount = len(set(cat_spList).intersection(sa_sp_codes))
                row[5] = 1
                row[9] = saspcount
            elif len(set(cat_spList).intersection(sa_no_sp_codes)) > 0  and len(set(cat_spList).intersection(sa_sp_codes)) == 0:
                print(f'{row[0]} has {len(set(cat_spList).intersection(sa_no_sp_codes))} salmon species records')
                row[5] = 2
                row[9] = 0
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[5] = 0
                row[9] = 0
        else:
            print(f'{row[0]} did not intersect the AWC ')
            row[1]=None
            row[2]=None
            row[3]=None
            row[4]=None
            row[5]=None
            row[6]=None
            row[7]=None
            row[8]=None
            row[9]=None
            norecslist.append(row[0])
            c2+=1
        cur.updateRow(row)
    del(row)
del(cur)

print(f'{c2} catchments did not have AWC species records')
print(norecslist)

['OBJECTID', 'cat_ID_con', 'region', 'MERGE_SRC', 'AWC_OID', 'AWC_CODE', 'SPECIES', 'LSTAGE', 'Shape_Length', 'Shape_Area', 'sp_ls', 'sa_sp_hab', 'sa_sp_count', 'mult_awc', 'AWC_CODE_qc']
****************************************************************************************************
Bristol_Bay_1003859 has 5 species_ls records ['S_s', 'P_p', 'CH_p', 'CO_p', 'K_p']
list without duplicate records ['K_p', 'CO_p', 'CH_p', 'S_s', 'P_p']
Bristol_Bay_1003859 has {'S_s'} salmon spawning records
****************************************************************************************************
Bristol_Bay_1004129 has 1 species_ls records ['S_p']
list without duplicate records ['S_p']
Bristol_Bay_1004129 has 1 salmon species records
****************************************************************************************************
Multiple awc records(2) for this catchment 
Bristol_Bay_1006858 has 4 species_ls records ['CO_p', 'S_s', 'CO_p', 'S_s']
list without duplicate records ['S_s', '

## Update using qc dictionaries

In [59]:
# cat_ID_con to corresponding AWC code from awcEventsArcs (2021) fc
awcQcDict = {
             }

# cat_ID_con to corresponding AWC code from AWCLakeEvents (2021) fc
awcLakesQcLakesDict = {
                       }

# Dictionary to relate catchments to awc using awc OIDs for those missed in spatial join
addQcDict = {
            }

print(f'QC Dictionaries set\n{"*"*100}')


QC Dictionaries set
****************************************************************************************************


Use AWC_OID codes to update

In [60]:
# List to store catchments that were did not get attributed in spatial join but were identified in initial review as awc catchments
adQC = []
# List to store catchments without spawning habitat
nosphab = []
sa_sp_codes = ['CO_s','S_s','K_s','CH_s','P_s']
sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']

fields = ['cat_ID_con','AWC_CODE_qc','sp_ls','sa_sp_hab','AWC_OID','AWC_CODE','mult_awc','LSTAGE','SPECIES','sa_sp_count']

c=1
with arcpy.da.UpdateCursor(awcjoin,fields) as cur:
    for row in cur:
        if row[0] in awcQcDict:
            print(f'\n{("*"*100)}')
            print(f'{c}. {row[0]} found in awc qc dictionary')
            updCode = awcQcDict[row[0]]
            if updCode is not None:
                if row[4] is  not None:
                    awcOidList = []
                    awcSpLS = []
                    awcOIDvals = row[4].split(",")
                    print(f' oid vals  = {awcOIDvals} - row4 = {row[4]}')
                    awcCodevals = row[5].split(",")
                    print(f'code vals = {awcCodevals}')
                    snum = len(row[5].split(","))
                    for i in range(snum):
                        if awcCodevals[i] == updCode:
                            awcoid = int(awcOIDvals[i])
                            print(f'awcCode value of i = {awcCodevals[i]} and updCode = {updCode}\nAWC_OID = {awcoid}')
                            print(f'Appending {awcOidDict[int(awcOIDvals[i])]} from OID dictionary')
                            awcSpLS.append(awcOidDict[int(awcOIDvals[i])])
                    updSpLS = str(awcSpLS).strip("[]").replace("'","")
                    print (f'Awc codes {row[1]} will be updated to {updCode}')
                    print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
                    if len(set(awcSpLS).intersection(sa_sp_codes)) >0:
                        print(f'{row[0]} has {len(set(awcSpLS).intersection(sa_sp_codes))} salmon spawning records')
                        saspcount = len(set(awcSpLS).intersection(sa_sp_codes))
                        row[3] = 1
                        row[9] = saspcount
                    elif len(set(awcSpLS).intersection(sa_no_sp_codes)) > 0  and len(set(awcSpLS).intersection(sa_sp_codes)) == 0:
                        print(f'{row[0]} has {len(set(awcSpLS).intersection(sa_no_sp_codes))} salmon species records')
                        row[3] = 2
                        row[9] = 0
                    else:
                        print(f'{row[0]} has no spawning habitat recorded')
                        nosphab.append(row[0])
                        row[3] = 0
                        row[9] = 0
                    row[1] = updCode
                    row[2] = updSpLS
                else:
                    sp_ls_missvals = []
                    print(f'{row[0]} missed during spatial join but identified during qc - must review again and add OBJECTIDs from awc to new keyval dictionary')
                    for val in addQcDict[row[0]]:
                        intval = int(val)
                        sp_ls_val = awcOidDict[intval]
                        sp_ls_missvals.append(sp_ls_val)
                    row[1] = updCode
                    # sp_ls
                    sp_ls_up = str(sp_ls_missvals).strip("[]").replace("\'","")
                    print(f'{sp_ls_up}')
                    row[2] = sp_ls_up
                    # sa_sp_hab
                    if len(set(sp_ls_missvals).intersection(sa_sp_codes)) >0:
                        print(f'{row[0]} has {len(set(sp_ls_missvals).intersection(sa_sp_codes))} salmon spawning records')
                        saspcount = len(set(sp_ls_missvals).intersection(sa_sp_codes))
                        row[3] = 1
                        row[9] = saspcount
                    elif len(set(sp_ls_missvals).intersection(sa_no_sp_codes)) > 0  and len(set(sp_ls_missvals).intersection(sa_sp_codes)) == 0:
                        print(f'{row[0]} has {len(set(sp_ls_missvals).intersection(sa_no_sp_codes))} salmon species records')
                        row[3] = 2
                        row[9] = 0
                    else:
                        print(f'{row[0]} has no spawning habitat recorded')
                        nosphab.append(row[0])
                        row[3] = 0
                        row[9] = 0
                    # AWC_OID
                    oids = str(addQcDict[row[0]]).strip("[]").replace("'","")
                    print(oids)
                    row[4] = oids
                    # AWC_CODE
                    awccode = str(awcQcDict[row[0]].replace("'",""))
                    print(awccode)
                    row[5] = awccode
                    adQC.append(row[0])
            else:
                print(f'{("*"*100)}\n')
                print(f'{row[0]} will have code removed {updCode}')
                row[1]=None
                row[2]=None
                row[3]=None
                row[4]=None
                row[5]=None
                row[6]=None
                row[7]=None
                row[8]=None
                row[9]=None

            c+=1
        elif row[0] in awcLakesQcLakesDict:
            print(f'{("*"*100)}\n')
            print(f'{c}. {row[0]} found in awc lakes qc dictionary')
            print(f'{row[0]} found in awc qc dictionary')
            updCode = awcLakesQcLakesDict[row[0]]
            updSpLS = str(awcLakesDict[updCode]).strip("[]").replace("'","")
            print (f'Awc codes {row[1]} will be updated to {updCode}')
            print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
            if len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))} salmon spawning records')
                saspcount = len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))
                row[3] = 1
                row[9] = saspcount
            elif len(set(awcLakesDict[updCode]).intersection(sa_no_sp_codes)) > 0  and len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) == 0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_no_sp_codes))} salmon species records')
                row[3] = 2
                row[9] = 0
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[3] = 0
                row[9] = 0
            row[1] = updCode
            row[2] = updSpLS
            c+=1
        else:
            pass
        cur.updateRow(row)
    del(row)
del(cur)


****************************************************************************************************
1. Cook_Inlet_75004300000100 found in awc qc dictionary
 oid vals  = ['20604', '20631', '20750', '20782', '20945', '20980', '21158', '21918', '21976', '22056', '22147', '22264', '22398'] - row4 = 20604,20631,20750,20782,20945,20980,21158,21918,21976,22056,22147,22264,22398
code vals = ['244-10-10010-2021', '244-10-10010', '244-10-10010-2021-3009', '244-10-10010', '244-10-10010', '244-10-10010-2021', '244-10-10010-2021', '244-10-10010-2021', '244-10-10010', '244-10-10010-2021', '244-10-10010-2021', '244-10-10010', '244-10-10010']
awcCode value of i = 244-10-10010-2021 and updCode = 244-10-10010-2021
AWC_OID = 20604
Appending CO_s from OID dictionary
awcCode value of i = 244-10-10010-2021 and updCode = 244-10-10010-2021
AWC_OID = 20980
Appending DV_r from OID dictionary
awcCode value of i = 244-10-10010-2021 and updCode = 244-10-10010-2021
AWC_OID = 21158
Appending CO_r from OID dictiona

## Join fields to tempsites using cat_ID_con
* Join Sp_Ls and sa_sp_hab to point fc of temperature sites

In [61]:
#sites = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_all_sites_sj_merge"
#sites = r"D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_sites_sj_merge"
print(sites)
sitefields = [f.name for f in arcpy.ListFields(sites)]
sitefields

T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_all_sites_sj_merge


['OBJECTID',
 'Shape',
 'SiteID',
 'Agnc_ID',
 'SourcNm',
 'Wtrbdy_n',
 'Verified',
 'Comments',
 'DM_Notes',
 'Region',
 'dist_coast_km',
 'NHDPlusID',
 'SourceFC',
 'GridCode',
 'AreaSqKm',
 'VPUID',
 'cat_ID',
 'cat_ID_txt',
 'cat_ID_con',
 'catID',
 'MERGE_SRC',
 'site_max_acc',
 'site_acc_sqKm',
 'Area_km2',
 'PathLength',
 'TerminalPa',
 'ArbolateSu',
 'StartFlag',
 'AreaSqKm_1',
 'LINKNO',
 'DSContArea',
 'USContArea',
 'DOUTEND',
 'DOUTSTART',
 'area_diff',
 'str_slope',
 'str_ord',
 'ds_dist_outlet',
 'lat',
 'lon',
 'SiteID_lower_2_cov_id',
 'Site',
 'Region_1',
 'str_ord_1',
 'str_slope_1',
 'dist_coast_km_1',
 'wtd_elev_MEAN',
 'cat_elev_MEAN',
 'wtd_slope_MEAN',
 'cat_slope_MEAN',
 'wtd_north_per',
 'wtd_wet_per',
 'wtd_lake_per',
 'wtd_glacier_per',
 'wtd_area_sqKM',
 'F2001',
 'F2002',
 'F2003',
 'F2004',
 'F2005',
 'F2006',
 'F2007',
 'F2008',
 'F2009',
 'F2010',
 'F2011',
 'F2012',
 'F2013',
 'F2014',
 'F2015',
 'F2016',
 'F2017',
 'F2018',
 'F2019',
 'site_lower',
 'c

In [62]:
from time import strftime
print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))

sitefields = [f.name for f in arcpy.ListFields(sites)]
awctextfields = ['sp_ls','AWC_CODE']
awcshortfields = ['sa_sp_hab','sa_sp_count']
sourceFieldsList = ['cat_ID_con','sp_ls','sa_sp_hab','sa_sp_count','AWC_CODE_qc']
updateFieldsList = ['cat_ID_con','sp_ls','sa_sp_hab','sa_sp_count','AWC_CODE']
# Use list comprehension to build a dictionary from a da SearchCursor
valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(awcjoin, sourceFieldsList)}

awcCatsDict = {}
for awctextfield in awctextfields:
    if awctextfield not in sitefields:
        print(f'adding field {awctextfield} to {sites}')
        arcpy.AddField_management(sites,awctextfield,'TEXT','','',500)
    else:
        print(f'{awctextfield} present')

for awcshortfield in awcshortfields:
    if awcshortfield not in sitefields:
        print(f'adding field {awcshortfield} to {sites}')
        arcpy.AddField_management(sites,awcshortfield,'SHORT')
    else:
        print(f'{awcshortfield} present')

with arcpy.da.UpdateCursor(sites,updateFieldsList) as cur:
    for row in cur:
        keyVal = row[0]
        if keyVal in valueDict:
            for n in range (1,len(sourceFieldsList)):
                row[n] = valueDict[keyVal][n-1]
            cur.updateRow(row)

del valueDict

print( "Finished script: " + strftime("%Y-%m-%d %H:%M:%S"))


Start script: 2022-02-26 12:24:09
sp_ls present
AWC_CODE present
sa_sp_hab present
sa_sp_count present
Finished script: 2022-02-26 12:24:29


# Updated to add to code above for catchments as well as sites.
No need to run unless re-using to update another dataset
~~## Update sp_ls_hab code as follows~~
* 1 for salmon spawning
* 2 for salmon species but no spawning
* 0 for awc records but neither salmon spawning nor any other combo of salmon sp_ls
* NULL no awc data at all~~

In [63]:
# sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
# sa_sp_codes = ['CO_s','S_s','K_s','CH_s','P_s']
# #sites = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_all_sites_sj_merge"
# sites = r"D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_sites_sj_merge"
# noSalmList = []
#
# with arcpy.da.UpdateCursor(sites, ['sp_ls','cat_ID_con','SiteID','sa_sp_hab']) as cur:
#     for row in cur:
#         if row[0] is not None:
#             print(f'{("*")*100}')
#             print(f'Site {row[2]} has {row[0]}')
#             sp_lsLs = row[0].replace(" ","").split(",")
#             print(sp_lsLs)
#             if len(set(sp_lsLs).intersection(sa_sp_codes)) > 0:
#                 print(f'{row[0]} has {len(set(sp_lsLs).intersection(sa_sp_codes))} salmon spawning records')
#                 row[3] = 1
#             elif len(set(sp_lsLs).intersection(sa_no_sp_codes)) > 0  and len(set(sp_lsLs).intersection(sa_sp_codes)) == 0:
#                 print(f'{row[0]} has {len(set(sp_lsLs).intersection(sa_no_sp_codes))} salmon species records')
#                 row[3] = 2
#             else:
#                 print(f'{row[0]} has no salmon spawning habitat or species presence #####################')
#                 noSalmList.append(row[1])
#                 row[3] = 0
#         cur.updateRow(row)
#     del(row)
# del(cur)
#
# print('complete')
# noSalmList
